# lembre-se de substituir /caminho/ pelo caminho do seu diretorio

In [ ]:
import folium
from branca.element import Figure
import re
import pandas as pd
import random

In [ ]:
coord = []
primeiro = 0
caminho_coordenadas = "/home/caminho/terceira-etapa_extrair_latitude/coordenadas.csv"
with open(caminho_coordenadas, 'r') as arquivo_csv:
    for line in arquivo_csv:
        line_limpa=[]
        line_de_trabalhod = line.strip()
        line_de_trabalhod = line_de_trabalhod.split(';')
        coordenada_limpa = line_de_trabalhod[1].split(',')
        line_limpa.append(line_de_trabalhod[0])
        line_limpa.append(coordenada_limpa[0])
        line_limpa.append(coordenada_limpa[1])
        coord.append(line_limpa)




In [ ]:
cainho_arquivo_bairros = "/home/caminho/segunda-etapa_analise_dados_medicos/dados_medicos_geolocator_bairros.csv"
dfa = pd.read_csv(cainho_arquivo_bairros, sep = ";")
lista_bairro = ['Santa Efigênia', 'Funcionários', 'Savassi', 'Santo Agostinho', 'Lourdes', 'Centro', 'Barro Preto']
dfa = dfa.query("bairros in @lista_bairro")
dfa = dfa.reset_index(drop=True)
dados_completos = dfa.values

In [ ]:
def achar_rua(nome_rua_buscar):

    
    padrao = re.compile(r'(?P<rua>.+?)\s*(\d+)')
    correspondencia = padrao.search(nome_rua_buscar)
    if correspondencia is not None:
        resultado = f'{correspondencia[0]}'
    else:
        resultado = f'{nome_rua_buscar}'

    return(resultado)

In [ ]:
#acha o nome do medico baseado na rua e na coordenada
nome_medico = []
for i in range(len(dados_completos)):
    aux_coor = []
    aux_achar = 0
    for j in range(len(coord)):
        nome_rua_geo = achar_rua(dados_completos[i][1]).lower()
        if nome_rua_geo == coord[j][0] and aux_achar == 0:
            aux_achar = 1
            
            aux_coor.append(dados_completos[i][0])
            aux_coor.append(float(coord[j][1]))
            aux_coor.append(float(coord[j][2]))
            aux_coor.append(dados_completos[i][5])
            aux_coor.append(nome_rua_geo)
            nome_medico.append(aux_coor)
    if aux_achar ==0:
        print(nome_rua_geo)


In [ ]:
'''
ajustar todos os endereços para dentro de uma area. no caso ajustaremos por bairos, visto que esses podem ter sido obtidos
errados devido geocoders.
'''
def def_maior_menor_y(pontos_da_area):
    maior_y = pontos_da_area[0][0]
    menor_y = pontos_da_area[0][0]
    aux_y_maior_menor = []
    for z in range(len(pontos_da_area)):
        if maior_y < pontos_da_area[z][0]:
            maior_y = pontos_da_area[z][0]
        if menor_y > pontos_da_area[z][0]:
            menor_y = pontos_da_area[z][0]
            
    aux_y_maior_menor = [maior_y, menor_y]
    return(aux_y_maior_menor)

def def_maior_menor_x(pontos_da_area):
    maior_x = pontos_da_area[0][1]
    menor_x = pontos_da_area[0][1]
    aux_x_maior_menor = []
    for z in range(len(pontos_da_area)):
        if maior_x < pontos_da_area[z][1]:
            maior_x = pontos_da_area[z][1]
        if menor_x > pontos_da_area[z][1]:
            menor_x = pontos_da_area[z][1]
    aux_x_maior_menor = [maior_x, menor_x]
    return(aux_x_maior_menor)

def def_areas_que_pode_pertencer(ponto_y, ponto_x, x_range, y_range):
    areas_provaveis = []
    for i in range(len(x_range)):
        if ((ponto_x <= x_range[i][0]) and ( ponto_x >= x_range[i][1])) and ((ponto_y <= y_range[i][0]) and ( ponto_y >= y_range[i][1])): 
            areas_provaveis.append(i)
    return(areas_provaveis)

def def_escolhe_entre_duas_areas(ponto_y, ponto_x, pontos_area_analise):
    contador=0
    for i in range(len(pontos_area_analise)):
        if i < len(pontos_area_analise)-1:
            x1, y1 = pontos_area_analise[i][1], pontos_area_analise[i][0]
            x2, y2 = pontos_area_analise[i+1][1], pontos_area_analise[i+1][0]
            if (ponto_y < y2)!= (ponto_y < y1):
                calculo_intercessao = x1 + ((ponto_y-y1)/(y2-y1))*(x2-x1)
                if ponto_x < calculo_intercessao:
                    contador += 1
        if i == len(pontos_area_analise) - 1:
            x1, y1 = pontos_area_analise[i][1], pontos_area_analise[i][0]
            x2, y2 = pontos_area_analise[0][1], pontos_area_analise[0][0]
            if (ponto_y < y2)!= (ponto_y < y1):
                calculo_intercessao = x1 + ((ponto_y-y1)/(y2-y1))*(x2-x1)
                if ponto_x < calculo_intercessao:
                    contador += 1
    return(contador%2)              
    
santa_efigenia = [
    [-19.927010760788278, -43.93229665454389],
    [-19.920698831259013, -43.930880766722616],
    [-19.920290543612147, -43.91700201269492],
    [-19.9297821668943, -43.92140344359006]
]
savassi = [
    [-19.930835265091094, -43.93916024967046],
    [-19.939749199418912, -43.94154929400086],
    [-19.941075365003428, -43.934258639802934],
    [-19.939286278579537, -43.92599094015791],
    [-19.929715604102903, -43.931908432873314]
]
funcionarios = [
    [-19.939286278579537, -43.92599094015791],
    [-19.9297821668943, -43.92140344359006],
    [-19.926613234958516, -43.93380772079743],
    [-19.928241746543815, -43.932914794679185]
]
lourdes = [
    [-19.939749199418912, -43.94154929400086],
    [-19.930835265091094, -43.93916024967046],
    [-19.929783603478892, -43.93197733555652],
    [-19.926613234958516, -43.93380772079743],
    [-19.92273716928243, -43.94511731698864],
    [-19.937394408664048, -43.949402649648526],
]
santo_agostinho = [
    [-19.937394408664048, -43.949402649648526],
    [-19.92273716928243, -43.94511731698864],
    [-19.929526685353064, -43.957871457764085],
    [-19.933852938681223, -43.955067187073844],
]
barro_preto = [
    [-19.92273716928243, -43.94511731698864],
    [-19.929526685353064, -43.957871457764085],
    [-19.924440736729164, -43.96002753632537],
    [-19.91794480822647, -43.95732386954611],
    [-19.917218538762782, -43.947968324182995]
]
bairro_centro = [
    [-19.917218538762782, -43.947968324182995],
    [-19.92273716928243, -43.94511731698864],
    [-19.927010760788278, -43.93229665454389],
    [-19.920698831259013, -43.930880766722616],
    [-19.913152578135485, -43.933323439303805],
    [-19.910643236269966, -43.94300227092327]
]
lista_de_areas = ['Santa Efigênia', 'Savassi', 'Funcionários', 'Lourdes', 'Santo Agostinho', 'Barro Preto', 'Centro']
todas_as_areas = [santa_efigenia, savassi, funcionarios, lourdes, santo_agostinho, barro_preto, bairro_centro]
x_maior_menor_cada_area = []
y_maior_menor_cada_area = []

for j in range(len(todas_as_areas)):
    aux_x_maior_menor = []
    aux_x_maior_menor = def_maior_menor_x(todas_as_areas[j])
    x_maior_menor_cada_area.append(aux_x_maior_menor)
    
    aux_y_maior_menor = []
    aux_y_maior_menor = def_maior_menor_y(todas_as_areas[j])
    y_maior_menor_cada_area.append(aux_y_maior_menor)
    
for i in range(len(nome_medico)):
    areas_que_podem_estar_dentro = []
    areas_que_podem_estar_dentro = def_areas_que_pode_pertencer(nome_medico[i][1], nome_medico[i][2], x_maior_menor_cada_area, y_maior_menor_cada_area)
    ponto_correto=[]
    # se o intervalo do ponto estiver dentro de 2 ou mais areas usa metodo Ray Casting para encontrar area correta
    if len(areas_que_podem_estar_dentro) > 1:
        for j in range(len(areas_que_podem_estar_dentro)):
            numero_area = areas_que_podem_estar_dentro[j]
            retorno = def_escolhe_entre_duas_areas(nome_medico[i][1], nome_medico[i][2], todas_as_areas[numero_area])
            if retorno == True:
                ponto_correto.append(areas_que_podem_estar_dentro[j])
        #se o intervalo do ponto estiver sido selecionado em um novo ponto correto
        #ajusta para o novo bairro baseado em ponto correto
        if len(ponto_correto) >= 1:
            areas_que_podem_estar_dentro = ponto_correto
            ponto_correto_ajustar = areas_que_podem_estar_dentro[0]
            nome_medico[i][3] = lista_de_areas[ponto_correto_ajustar]
    #se o intervalo do ponto estiver dentro de apenas 1 area ajusta para a nova area baseado 
    if len(areas_que_podem_estar_dentro) == 1:
        ponto_correto_ajustar = areas_que_podem_estar_dentro[0]
        nome_medico[i][3] = lista_de_areas[ponto_correto_ajustar]



In [ ]:
coordenadas_centro = [-19.92273716928243, -43.94511731698864]
figmap = Figure(height=300, width=400)
colors = {'Santa Efigênia': 'purple',
 'Santo Agostinho': 'red',
 'Lourdes': 'cadetblue',
 'Savassi': 'orange',
 'Funcionários':'pink',
 'Barro Preto': 'beige',
 'Centro':'green'}

centro = folium.Map(location = coordenadas_centro, zoom_start = 14, tiles='cartodbpositron')
nomes_ruas_add_no_mapa = []

for i in range(len(nome_medico)):
    nome = nome_medico[i][0]
    coord_sendo_analisada = str(nome_medico[i][1]) + str(nome_medico[i][2])
    if coord_sendo_analisada in nomes_ruas_add_no_mapa:
        rand1 = (float(random.randint(1, 7))/10 ** 4)*0.5
        rand2 = (float(random.randint(1, 7))/10 ** 4)*0.5
        localizacao = [float(nome_medico[i][1]) + rand1, float(nome_medico[i][2]) + rand2]
    else:
        nomes_ruas_add_no_mapa.append(coord_sendo_analisada)
        localizacao = [float(nome_medico[i][1]) , float(nome_medico[i][2])]
    cor_chave = str(nome_medico[i][3])
    if cor_chave in colors:
        icone = folium.Icon(color=colors[cor_chave])

    else:
        print(cor_chave)
        icone = folium.Icon(color='gray')
    pin = folium.Marker(location = localizacao, popup = nome, icon = icone)
    pin.add_to(centro)

folium.Polygon(locations=(santa_efigenia, savassi, funcionarios, lourdes, santo_agostinho, barro_preto, bairro_centro), color='blue', fill=True, fill_color='blue', fill_opacity=0.1).add_to(centro)                        
nome = "Praça Raul Soares"
icone = folium.Icon(color='black')
pin = folium.Marker(location = coordenadas_centro, popup = nome, icon=icone)
pin.add_to(centro)
figmap.add_child(centro)

In [ ]:
cainho_arquivo_bairros = "/home/caminho/quarta-etapa_criar_mapa/mapa_centro.html"
figmap.save(cainho_arquivo_bairros)